In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import io
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import math
from numpy import dot
from numpy.linalg import norm

In [43]:
#Edit the path and Keep the query document and database documents in that directory.
#Run the script

path = r'C:\Users\Darshan\Music\chinmay_hegde\hw1'
os.chdir(path)

In [44]:
def preProcessing(texts):
    processed_sentences = []
    for text in sent_tokenize(texts):
        
        #Removing all special characters'
        processed_feature = re.sub(r'\W', ' ', text)
        
        #Removinf single characters
        processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
        
        #Removing digits from the text
        processed_feature = re.sub('\d+',' ',processed_feature)
        
        # Substituting multiple spaces with single space
        processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

        # Converting to Lowercase
        processed_feature = processed_feature.lower()
        
        #Clearing out whitespaces
        processed_feature = processed_feature.rstrip().lstrip()

        # Storing each processed sent in the list
        processed_sentences.append(processed_feature)
        
    processed_sent = ' '.join(processed_sentences)
    return processed_sent

In [45]:
#Preprocessing the given documents including the query document and renaming with an extension _pp at the end in the same directory

no_of_docs = 0
for file in os.listdir('.'):
    file_name,file_ext = os.path.splitext(file)
    if(file_ext == '.txt'):
        file_content = open(file,'r',encoding='utf-8').readlines()
        processed_file_content = preProcessing(str(file_content))
        fp = open(f'{file_name}_pp{file_ext}','w',encoding='utf-8')
        fp.write(processed_file_content)
        fp.close()
        no_of_docs +=1

In [46]:
#Calculating Unique words vector from all given database documents


unique_words = set()
for file in os.listdir('.'):
    file_name,file_ext = os.path.splitext(file)
    if('_pp' in file_name and (not 'query' in file_name)):
        fp = open(file,'r',encoding='utf-8').readlines()
        file_content = str(fp[0])
        sentences = sent_tokenize(file_content)
        for sent in sentences:
            words = word_tokenize(sent)
            for w in words:
                if not (w in unique_words):
                    unique_words.add(w)
                    
uw = np.array(list(unique_words))


In [51]:

#Calculating vector for each document

dict_ = {}
doc = 1
v_words = {i:{} for i in range(1,no_of_docs+1)}
for file in os.listdir('.'):
    file_name,file_ext = os.path.splitext(file)
    if('_pp' in file_name):
        vector = [0]*len(uw)
        temp = {}
        fp = open(file,'r',encoding='utf-8').readlines()
        file_content = str(fp[0])
        sentences = sent_tokenize(file_content)
        for sent in sentences:
            words = word_tokenize(sent)
            for w in words:
                
                if(w in temp.keys()):
                    temp[w]+=1
                else:
                    temp[w] = 1
                    
                if(w in unique_words):
                    vector[list(unique_words).index(w)] = 1
        v_words[doc] = temp
        dict_[doc] = np.array(vector)
        doc+=1

        
#Part 4a

for i in range(1,no_of_docs+1):
    print(f'Document {i} vector of words is as follows\n\n ')
    print(v_words[i])
    print('\n')
    

#This part is representing document i vector form

#for key,val in dict_.items():
#    print(key,dict_[key]) 


6
Document 1 vector of words is as follows

 
{'java': 8, 'is': 7, 'an': 2, 'island': 5, 'of': 15, 'indonesia': 4, 'with': 1, 'population': 2, 'over': 1, 'million': 2, 'home': 1, 'to': 1, 'percent': 1, 'the': 18, 'indonesian': 4, 'and': 5, 'most': 3, 'populous': 1, 'on': 5, 'earth': 1, 'capital': 1, 'city': 1, 'jakarta': 1, 'located': 1, 'western': 1, 'much': 1, 'history': 1, 'took': 1, 'place': 1, 'it': 2, 'was': 1, 'center': 1, 'powerful': 1, 'hindu': 1, 'buddhist': 1, 'empires': 1, 'islamic': 1, 'sultanates': 1, 'core': 1, 'colonial': 1, 'dutch': 1, 'east': 2, 'indies': 1, 'n': 1, 'formed': 1, 'mostly': 1, 'as': 2, 'result': 1, 'volcanic': 2, 'eruptions': 1, 'thirteenth': 1, 'largest': 2, 'in': 3, 'world': 1, 'fifth': 1, 'a': 1, 'chain': 1, 'mountains': 1, 'forms': 1, 'west': 1, 'spine': 1, 'along': 1, 'three': 1, 'main': 1, 'languages': 1, 'are': 3, 'spoken': 1, 'javanese': 2, 'sundanese': 1, 'madurese': 1, 'these': 1, 'dominant': 1, 'native': 1, 'language': 3, 'about': 1, 'people'

In [58]:



#Initializing the dataframe with column names

df = pd.DataFrame()
names = ['Words','D1_TF','D2_TF','D3_TF','D4_TF','D5_TF','D_QUERY_TF','IDF','TFIDF_D1','TFIDF_D2','TFIDF_D3','TFIDF_D4','TFIDF_D5','TFIDF_D_QUERY']
df['Words'] = list(uw)
for i in range(1,len(names)):
    df[names[i]] = [0.0]*len(uw)


#Calculating TF for each word in every document as well as query 
#Part 4b
    
doc_length = []
num = 1
for file in os.listdir('.'):
    file_name,file_ext = os.path.splitext(file)
    if('_pp' in file_name):
        #print(file_name)
        fp = open(file,'r',encoding='utf-8').readlines()
        file_content = str(fp[0])
        
        dict_w = {}
        length = 0
        
        sentences = sent_tokenize(file_content)
        
        for sent in sentences:
            words = word_tokenize(sent)
            length += len(words)
            for w in words:
                if(w in dict_w.keys()):
                    dict_w[w] +=1
                else:
                    dict_w[w] = 1
                    
        doc_length.append(length)        
        wc = np.array(list(dict_w.values())) / length
        vector = [0]*len(uw)
        
        j = 0
        for i in range(len(dict_[num])):
            if(dict_[num][i]==1):
                vector[i] = wc[j]
                j+=1
                
        if('_query' in file_name):
            df['D_QUERY_TF'] = vector
        else:
            file_ = file_name.split('_')[0]
            df[file_.upper()+'_TF'] = vector
            
        num+=1
      
    #print(df)

In [54]:
#Calculating IDF for each word

df1 = df.iloc[:,1:6]
math_ = []
for i in range(len(df1)): 
    count = 0
    for j in df1.iloc[i]:
        if(j > 0):
            count +=1
    math_.append(math.log((len(doc_length)-1)/(count)))

df['IDF'] = math_

df

,Words,D1_TF,D2_TF,D3_TF,D4_TF,D5_TF,D_QUERY_TF,IDF,TFIDF_D1,TFIDF_D2,TFIDF_D3,TFIDF_D4,TFIDF_D5,TFIDF_D_QUERY
0,wet,0.000000,0.000000,0.000000,0.026616,0.000000,0.000000,1.609438,0.0,0.0,0.0,0.0,0.0,0.0
1,than,0.000000,0.000000,0.000000,0.038023,0.053333,0.000000,0.916291,0.0,0.0,0.0,0.0,0.0,0.0
2,ashore,0.000000,0.000000,0.003311,0.000000,0.000000,0.000000,1.609438,0.0,0.0,0.0,0.0,0.0,0.0
3,out,0.000000,0.000000,0.026490,0.000000,0.000000,0.000000,1.609438,0.0,0.0,0.0,0.0,0.0,0.0
4,n,0.045198,0.033816,0.023179,0.007605,0.017778,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
5,control,0.000000,0.000000,0.000000,0.030418,0.000000,0.000000,1.609438,0.0,0.0,0.0,0.0,0.0,0.0
6,co,0.000000,0.028986,0.000000,0.000000,0.000000,0.000000,1.609438,0.0,0.0,0.0,0.0,0.0,0.0
7,note,0.000000,0.000000,0.000000,0.007605,0.000000,0.000000,1.609438,0.0,0.0,0.0,0.0,0.0,0.0
8,clove,0.000000,0.000000,0.000000,0.003802,0.000000,0.000000,1.609438,0.0,0.0,0.0,0.0,0.0,0.0
9,indies,0.039548,0.000000,0.000000,0.000000,0.000000,0.000000,1.609438,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
#Calculating TF-IDF for each word

names = ['TFIDF_D1','TFIDF_D2','TFIDF_D3','TFIDF_D4','TFIDF_D5','TFIDF_D_QUERY']
doc_names = ['D1_TF','D2_TF','D3_TF','D4_TF','D5_TF','D_QUERY_TF']
for n in range(0,len(names)):
    df[names[n]] = (np.array(df[doc_names[n]]) * np.array(df['IDF']))


#Saving it in a file

columns = ['Words','D1_TF','D2_TF','D3_TF','D4_TF','D5_TF','D_QUERY_TF','IDF','TFIDF_D1','TFIDF_D2','TFIDF_D3','TFIDF_D4','TFIDF_D5','TFIDF_D_QUERY']
df.to_csv('Data.csv',header=True,columns = columns,encoding='utf-8',sep=',')


In [56]:
#Calculating the cosine similarity
#Part 4c and 4d


names = ['TFIDF_D1','TFIDF_D2','TFIDF_D3','TFIDF_D4','TFIDF_D5']
max_sim = -float('inf')
b = np.array(df['TFIDF_D_QUERY'])
name = ''
print('Cosine similarity of Doc1 with Query Doc is given as follows\n')
for i in range(len(names)):
    a = np.array(df[names[i]])
    cos_sim = dot(a,b)/(norm(a)*norm(b))
    print('{:.4f}'.format(cos_sim),names[i],'TFIDF_D_QUERY')
    if(cos_sim > max_sim):
        max_sim = cos_sim
        name = names[i]

print('\n\n')
max_sim = max_sim * 100
print(f'The most similar document for query is found as {name} and similarity percentage is {max_sim:.2f}')

Cosine similarity of Doc1 with Query Doc is given as follows

0.0000 TFIDF_D1 TFIDF_D_QUERY
0.0000 TFIDF_D2 TFIDF_D_QUERY
0.0000 TFIDF_D3 TFIDF_D_QUERY
0.0736 TFIDF_D4 TFIDF_D_QUERY
0.0000 TFIDF_D5 TFIDF_D_QUERY



The most similar document for query is found as TFIDF_D4 and similarity percentage is 7.36
